In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
sns.set_color_codes("muted")
import matplotlib.pyplot as plt
import gc 

% matplotlib inline

In [2]:
df = pd.read_csv('data/train_date.csv.zip', index_col=0, nrows=10000)

In [3]:
response = pd.read_csv('data/train_numeric.csv.zip', index_col=0, usecols=[0, 969])

/home/li/miniconda3/envs/kaggle/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
print(df.shape)
df.dropna(axis=0, how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)
print(df.shape)

(10000, 1156)
(9997, 1145)


In [5]:
# find the first feature in each station
cols = df.columns.tolist()
stations = []
a = list(set(x.split('_')[1] for x in cols))
for c in cols:
    tmp = c.split('_')[1]
    if (tmp not in stations) and (tmp in a):
        stations.append(c)
        a.remove(tmp)        
del a, tmp, c, cols
gc.collect()
stations.insert(0, 'Id')
print(stations)

['Id', 'L0_S0_D1', 'L0_S1_D26', 'L0_S2_D34', 'L0_S3_D70', 'L0_S4_D106', 'L0_S5_D115', 'L0_S6_D120', 'L0_S7_D137', 'L0_S8_D145', 'L0_S9_D152', 'L0_S10_D216', 'L0_S11_D280', 'L0_S12_D331', 'L0_S13_D355', 'L0_S14_D360', 'L0_S15_D395', 'L0_S16_D423', 'L0_S17_D432', 'L0_S18_D437', 'L0_S19_D454', 'L0_S20_D462', 'L0_S21_D469', 'L0_S22_D543', 'L0_S23_D617', 'L1_S24_D677', 'L1_S25_D1854', 'L2_S26_D3037', 'L2_S27_D3130', 'L2_S28_D3223', 'L3_S29_D3316', 'L3_S30_D3496', 'L3_S31_D3836', 'L3_S32_D3852', 'L3_S33_D3856', 'L3_S34_D3875', 'L3_S35_D3886', 'L3_S36_D3919', 'L3_S37_D3942', 'L3_S38_D3953', 'L3_S39_D3966', 'L3_S40_D3981', 'L3_S41_D3997', 'L3_S43_D4062', 'L3_S44_D4101', 'L3_S45_D4125', 'L3_S47_D4140', 'L3_S48_D4194', 'L3_S49_D4208', 'L3_S50_D4242', 'L3_S51_D4255']


In [6]:
df = pd.read_csv('data/train_date.csv.zip', index_col=0, usecols=stations)

/home/li/miniconda3/envs/kaggle/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
df['date_start'] = df.min(axis=1)
df['date_end'] = df.max(axis=1)
df['duration'] = df.date_end - df.date_end

In [8]:
stations.remove('Id')
stations.append('date_end')
stations.append('duration')

In [9]:
df.drop(stations, axis=1, inplace=True)

In [18]:
df = df.merge(response, left_index=True, right_index=True)

In [21]:
df = df.reset_index()

In [50]:
df.sort_values(['date_start', 'Id'], inplace=True)

In [51]:
date_start = df.date_start.unique()

In [52]:
df.dropna(subset=['date_start'], axis=0, inplace=True)

In [53]:
date = df.groupby('date_start')['Response'].agg(['mean', 'sum', 'count']).sort_values(by=['sum', 'mean', 'count'], ascending=False).reset_index()

In [154]:
predicting_power = {}

for n in range(400):
    
    x = df.loc[df.date_start==date.date_start[n]].copy()

    x['diff'] = x['Id'].diff().fillna(99999999).astype(int)
    x['corr'] = x.Response.rolling(2).sum().fillna(99999999).astype(int)
    x = x.iloc[1:]

    if 1 in x['diff'].values:
        x.groupby('diff').corr.agg(['mean', 'count', 'sum']).loc[1]
        a.append(len(x[(x['diff']==1) & (x['corr']==2)])/len(x))
        predicting_power[n] = [date.date_start[n], len(x[(x['diff']==1) & (x['corr']==2)])/len(x), len(x[(x['diff']==1) & (x['corr']==2)]), len(x)]

predicting_power = pd.DataFrame.from_dict(predicting_power, orient='index')

predicting_power.columns = ['start_date', 'error_rate', 'error_count', 'total_count']

rate = predicting_power.error_count.sum()/predicting_power.total_count.sum()

In [155]:
rate

0.031465247338760174